In [2]:
import requests
import xml.etree.ElementTree as ET

auth_key = "c806e29e6e85e4ed20d0563d93ebf0bd1f272962598876ea14c651141d8bb18d"
url = f"http://data4library.kr/api/loanItemSrch?authKey={auth_key}&pageSize=5000"

# API 호출 및 XML 데이터 파싱
response = requests.get(url)
response.raise_for_status()
root = ET.fromstring(response.content)

# 'docs' 요소를 찾고, 그 하위의 'doc' 요소에서 ISBN 번호 추출
docs = root.find("docs")
if docs is not None:
    isbn_list = [doc.find("isbn13").text.strip() for doc in docs.findall("doc") if doc.find("isbn13") is not None]
else:
    isbn_list = []

print(isbn_list)

['9788972756194', '9788936433673', '9788956604992', '9788973374113', '9788976491114', '9788936433598', '9788996991342', '9788936434267', '9788990611260', '9788937473135', '9788934972464', '9788986621136', '9788965744023', '9788936434120', '9788934908067', '9788935652792', '9788982814471', '9788957090251', '9788965700609', '9788954608640', '9788956609959', '9788937838750', '9788995151204', '9788937853470', '9788954622035', '9788937889714', '9788936446819', '9788984370753', '9791161571188', '9791195522125', '9788936433871', '9788934939603', '9788937848247', '9791195794799', '9788963700342', '9788949190020', '9788972753698', '9788986621105', '9788949110004', '9788983920690', '9788937842184', '9788932916194', '9788936433697', '9788954427173', '9788984371026', '9788965700036', '9788956607030', '9788901033518', '9791196025809', '9791162332542', '9788970123691', '9788965744030', '9788986565065', '9788989863786', '9788970940199', '9791165341909', '9788937842191', '9788970127248', '978898662119

In [3]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm

# API 키를 문자열로 정의
auth_key = "c806e29e6e85e4ed20d0563d93ebf0bd1f272962598876ea14c651141d8bb18d"

# 결과 저장을 위한 빈 리스트
book_info_data = []
loan_info_data = []
loan_grp_data = []
keywords_data = []
co_loan_books_data = []
mania_rec_books_data = []
reader_rec_books_data = []

# 각 ISBN에 대해 API 호출 및 데이터 수집
for isbn in tqdm.tqdm(isbn_list, desc="Processing ISBNs"):
    url = f"http://data4library.kr/api/usageAnalysisList?authKey={auth_key}&isbn13={isbn}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외를 발생시킵니다.

        # XML 응답 파싱
        root = ET.fromstring(response.text)

        # 가장 처음 나오는 도서 정보 가져오기
        first_book = root.find(".//book")
        if first_book is not None:
            bookname = first_book.find("bookname").text if first_book.find("bookname") is not None else "N/A"
        else:
            bookname = "N/A"

        # 도서 서지 정보 데이터 추출
        for book in root.findall(".//book"):
            try:
                authors = book.find("authors").text if book.find("authors") is not None else "N/A"
                publisher = book.find("publisher").text if book.find("publisher") is not None else "N/A"
                book_image_url = book.find("bookImageURL").text if book.find("bookImageURL") is not None else "N/A"
                description = book.find("description").text if book.find("description") is not None else "N/A"
                publication_year = book.find("publication_year").text if book.find("publication_year") is not None else "N/A"
                isbn13 = book.find("isbn13").text if book.find("isbn13") is not None else "N/A"
                vol = book.find("vol").text if book.find("vol") is not None else "N/A"
                class_no = book.find("class_no").text if book.find("class_no") is not None else "N/A"
                class_nm = book.find("class_nm").text if book.find("class_nm") is not None else "N/A"
                loan_cnt = int(book.find("loanCnt").text) if book.find("loanCnt") is not None else 0
                book_info_data.append({"도서명": bookname, "저자명": authors, "출판사": publisher,
                                       "책표지 URL": book_image_url, "책소개": description, "출판년도": publication_year,
                                       "13 자리 ISBN": isbn13, "권": vol, "주제분류": class_no, "주제분류명": class_nm, "대출건수": loan_cnt})
            except Exception as e:
                print(f"Error processing book data: {e}")

        # 대출 정보 데이터 추출
        for loan in root.findall(".//loan"):
            try:
                month = loan.find("month").text if loan.find("month") is not None else "N/A"
                loan_cnt = int(loan.find("loanCnt").text) if loan.find("loanCnt") is not None else 0
                ranking = int(loan.find("ranking").text) if loan.find("ranking") is not None else 0
                loan_info_data.append({"도서명": bookname, "대출년월": month, "대출건수": loan_cnt, "순위": ranking})
            except Exception as e:
                print(f"Error processing loan data: {e}")

        # 다대출 이용자 그룹 데이터 추출
        first_class_nm = root.find(".//book/class_nm").text if root.find(".//book/class_nm") is not None else "N/A"

        for grp in root.findall(".//loanGrps/loanGrp"):
            try:
                age = grp.find("age").text if grp.find("age") is not None else "N/A"
                gender = grp.find("gender").text if grp.find("gender") is not None else "N/A"
                loan_cnt = int(grp.find("loanCnt").text) if grp.find("loanCnt") is not None else 0
                ranking = int(grp.find("ranking").text) if grp.find("ranking") is not None else 0
                
                # 여기서 first_class_nm을 사용하여 데이터 프레임에 추가
                loan_grp_data.append({
                    "도서명": bookname,
                    "연령": age,
                    "성별": gender,
                    "대출건수": loan_cnt,
                    "순위": ranking,
                    "주제분류명": first_class_nm
                })
            except Exception as e:
                print(f"Error processing loan group data: {e}")
        # 키워드 정보 데이터 추출
        for keyword in root.findall(".//keywords/keyword"):
            try:
                word = keyword.find("word").text if keyword.find("word") is not None else "N/A"
                weight = float(keyword.find("weight").text) if keyword.find("weight") is not None else 0.0
                keywords_data.append({"도서명": bookname, "키워드": word, "가중치": weight})
            except Exception as e:
                print(f"Error processing keyword data: {e}")

        # 함께 대출된 도서 데이터 추출
        for co_book in root.findall(".//coLoanBooks/book"):
            try:
                co_bookname = co_book.find("bookname").text if co_book.find("bookname") is not None else "N/A"
                co_authors = co_book.find("authors").text if co_book.find("authors") is not None else "N/A"
                co_publisher = co_book.find("publisher").text if co_book.find("publisher") is not None else "N/A"
                co_publication_year = co_book.find("publication_year").text if co_book.find("publication_year") is not None else "N/A"
                co_isbn13 = co_book.find("isbn13").text if co_book.find("isbn13") is not None else "N/A"
                co_vol = co_book.find("vol").text if co_book.find("vol") is not None else "N/A"
                co_loan_cnt = int(co_book.find("loanCnt").text) if co_book.find("loanCnt") is not None else 0
                co_loan_books_data.append({"도서명": bookname, "함께 대출된 도서명": co_bookname, "저자명": co_authors, "출판사": co_publisher,
                                           "출판년도": co_publication_year, "13 자리 ISBN": co_isbn13, "권": co_vol, "대출건수": co_loan_cnt})
            except Exception as e:
                print(f"Error processing co-loan books data: {e}")

        # 마니아 추천 도서 데이터 추출
        for mania_book in root.findall(".//maniaRecBooks/book"):
            try:
                mania_bookname = mania_book.find("bookname").text if mania_book.find("bookname") is not None else "N/A"
                mania_authors = mania_book.find("authors").text if mania_book.find("authors") is not None else "N/A"
                mania_publisher = mania_book.find("publisher").text if mania_book.find("publisher") is not None else "N/A"
                mania_publication_year = mania_book.find("publication_year").text if mania_book.find("publication_year") is not None else "N/A"
                mania_isbn13 = mania_book.find("isbn13").text if mania_book.find("isbn13") is not None else "N/A"
                mania_vol = mania_book.find("vol").text if mania_book.find("vol") is not None else "N/A"
                mania_rec_books_data.append({"도서명": bookname, "마니아 추천 도서명": mania_bookname, "저자명": mania_authors, "출판사": mania_publisher,
                                              "출판년도": mania_publication_year, "13 자리 ISBN": mania_isbn13, "권": mania_vol})
            except Exception as e:
                print(f"Error processing mania rec books data: {e}")

        # 다독자 추천 도서 데이터 추출
        for reader_book in root.findall(".//readerRecBooks/book"):
            try:
                reader_bookname = reader_book.find("bookname").text if reader_book.find("bookname") is not None else "N/A"
                reader_authors = reader_book.find("authors").text if reader_book.find("authors") is not None else "N/A"
                reader_publisher = reader_book.find("publisher").text if reader_book.find("publisher") is not None else "N/A"
                reader_publication_year = reader_book.find("publication_year").text if reader_book.find("publication_year") is not None else "N/A"
                reader_isbn13 = reader_book.find("isbn13").text if reader_book.find("isbn13") is not None else "N/A"
                reader_vol = reader_book.find("vol").text if reader_book.find("vol") is not None else "N/A"
                reader_rec_books_data.append({"도서명": bookname, "다독자 추천 도서명": reader_bookname, "저자명": reader_authors, "출판사": reader_publisher,
                                               "출판년도": reader_publication_year, "13 자리 ISBN": reader_isbn13, "권": reader_vol})
            except Exception as e:
                print(f"Error processing reader rec books data: {e}")

    except requests.RequestException as e:
        print(f"Request failed: {e}")

# 데이터 프레임 생성
df_book_info = pd.DataFrame(book_info_data)
df_loan_info = pd.DataFrame(loan_info_data)
df_loan_grp = pd.DataFrame(loan_grp_data)
df_keywords = pd.DataFrame(keywords_data)
df_co_loan_books = pd.DataFrame(co_loan_books_data)
df_mania_rec_books = pd.DataFrame(mania_rec_books_data)
df_reader_rec_books = pd.DataFrame(reader_rec_books_data)

# Excel 파일로 저장
with pd.ExcelWriter("book_analysis.xlsx") as writer:
    df_book_info.to_excel(writer, sheet_name="도서 서지 정보", index=False)
    df_loan_info.to_excel(writer, sheet_name="대출 정보", index=False)
    df_loan_grp.to_excel(writer, sheet_name="다대출 이용자 그룹", index=False)
    df_keywords.to_excel(writer, sheet_name="키워드 정보", index=False)
    df_co_loan_books.to_excel(writer, sheet_name="함께 대출된 도서", index=False)
    df_mania_rec_books.to_excel(writer, sheet_name="마니아 추천 도서", index=False)
    df_reader_rec_books.to_excel(writer, sheet_name="다독자 추천 도서", index=False)

print("Data has been successfully saved to book_analysis.xlsx")


Processing ISBNs: 100%|█████████████████████| 5000/5000 [56:19<00:00,  1.48it/s]


Data has been successfully saved to book_analysis.xlsx
